In [1]:
import subprocess
import shlex
import pandas as pd
import numpy as np
from astropy.table import Table
from astropy.table import Column
import os
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.ticker import MultipleLocator
import glob

In [2]:
basedir = "/home/xhall/Documents/"

RedshiftClass = Table.from_pandas(pd.read_csv(basedir + "NewZTF/ML_sample_snid100.csv"))

In [3]:
source = basedir + "NewZTF/SNIDoutput/"
output = basedir + "NewZTF/Images/"

In [4]:
def read_tables(files):
    matches_files = files[0:len(files)-1]
    spectra = Table.read(files[-1], format = "ascii", names = ["wavelength", "flux"])
    matches = []
    for i in matches_files:
        input_data = open(i,'r').readlines()[0].split()
        row = [[int(input_data[3][0]), input_data[4],input_data[5][1::],float(input_data[-3]),float(input_data[-1])]]
        row.append(Table.read(i, format = "ascii", names = ["redshifted_wavelength", "flux"]))
        matches.append(row)
    return matches, spectra

In [5]:
def plot_box_spec(wave, flux): 
    flux_plot = np.repeat(flux, 2)
    wv_plot = wave.copy()
    wv_plot[:-1] += np.diff(wave)/2
    wv_plot = np.append(wave[0]-(wave[1]-wave[0])/2, 
                        np.append(np.repeat(wv_plot[0:-1], 2), 
                                  wave[-1]+(wave[-1]-wave[-2])/2))
    
    return wv_plot, flux_plot

In [6]:
def specplot(x,y,xi,yi,snid_type,fname,output,best_num, z_template, z_template_unc, z_snid):
    fig, ax = plt.subplots(figsize=(8,4.5))
    ax.plot(xi,yi,color='#32384D',alpha=0.5,
             label='New SN')
    ax.plot(x,y,color='#217CA3',
             label='SNID template', lw=3)
    ax.plot(x[-3],y[-3],color='white',lw=0,
             label=r'$z_\mathrm{} = $ {:.3f}$\,\pm\,${:.3f}'.format("{SNID}", z_template, z_template_unc))
    ax.plot(x[-3],y[-3],color='#217CA3', lw=3)
    ax.set_xlabel(r'Rest Frame Wavelength ($\mathrm{\AA}$)', fontsize=17)
    ax.set_ylabel('Relative Flux', fontsize=17)
    ax.tick_params(which='both',labelsize=15)
    
    ax.grid(axis='x', color='0.7', ls=':')
    ax.xaxis.set_minor_locator(MultipleLocator(250))
    ax.set_yticklabels([])

    
    ax.text(0.105, 0.955, 'SNID type: ', 
            va='center',
            fontsize=15, transform=plt.gcf().transFigure)
    ax.text(0.245, 0.955, snid_type, 
            color='#217CA3', weight='bold', va='center',
            fontsize=23, transform=plt.gcf().transFigure)

    ax.text(0.78, 0.955, r'$z_\mathrm{} = ${:.4f}'.format("{SN}", z_snid), 
            va='center',
            fontsize=15, transform=plt.gcf().transFigure)
    
    
    ax.legend(fancybox=True)
    fig.subplots_adjust(left=0.055,right=0.99,top=0.925,bottom=0.145)
    fig.savefig(output + 'snidfits_emclip_' + fname + "_" + str(best_num) + '.png', dpi = 600)
    plt.close(fig)


In [7]:
def plot_best_5(source, output, spectra_name, z_snid):
    source_folder = source + spectra_name
    
    files = np.sort(glob.glob(source_folder+"/*.dat"))
    matches, spectra = read_tables(files)
    
    for i in matches:
        z = i[0][3]
        snid_type = i[0][2][:-1]
        
        xi, yi = plot_box_spec(spectra["wavelength"], spectra["flux"])
        xi /= (1+z)
        yi /= (1+z)
        x, y = i[1]["redshifted_wavelength"] / (1+z), i[1]["flux"] /(1+z)
        specplot(x,y,xi,yi,snid_type,spectra_name,output,i[0][0], z, i[0][4], z_snid)

In [8]:
for i in RedshiftClass:
    spectra_name = i["Version"].split(".")[0]
    z_snid = i["z_snid"]
    plot_best_5(source,output,spectra_name,z_snid)
    break